In [32]:
!pip install arxiv
!pip install fitz

In [33]:
import arxiv
import os
import fitz
import json

In [26]:
# connect to drive
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Download arXiv dataset regarding Model II regression

In [31]:
# Download arXiv dataset

# Configuration
search_query = "Model II regression"  # Words in the abstract
category = "math.ST"  # arXiv category, e.g., computer science (machine learning)
max_results = 50  # Limit for demonstration
output_dir = "/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs"  # Directory to save PDFs

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Formulate the search query with category
query = f"{search_query} cat:{category}"

# Search for papers
search = arxiv.Search(
    query=query,
    max_results=max_results,
    sort_by=arxiv.SortCriterion.Relevance,
)

# Download PDFs
for result in search.results():
    #print(f"Downloading: {result.title}")
    pdf_path = os.path.join(output_dir, f"{result.entry_id.split('/')[-1]}.pdf")
    try:
        result.download_pdf(dirpath=output_dir)
        #print(f"Saved to {pdf_path}")
    except arxiv.HTTPError as e:
        if e.code == 404:
            print(f"PDF not found for {result.entry_id}, skipping...")
        else:
            print(f"Error downloading {result.entry_id}: {e}")
    except Exception as e: # Catching general exception
        print(f"An unexpected error occurred while downloading {result.entry_id}: {e}")

print(f"PDFs saved in {output_dir}")

<ipython-input-31-3aa1f2dff22e>:23: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  for result in search.results():


An unexpected error occurred while downloading http://arxiv.org/abs/1909.10739v3: HTTP Error 404: Not Found
PDFs saved in /content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs


## Extract Text from PDFs

extract text from the pdfs - should this be saved as a json or txt?
also run on hpc this is a lot

In [34]:
# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    # Open the PDF file
    doc = fitz.open(pdf_path)
    text = ""

    # Extract text from each page
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text()

    return text

# Example usage
# pdf_path = "arxiv_pdf/2101.00001.pdf"  # Replace with your downloaded PDF
# paper_text = extract_text_from_pdf(pdf_path)
# print(paper_text[:1000])  # Print the first 1000 characters of the extracted text

# extract text from all pdfs and save each in a json file
pdf_dir = "/content/drive/MyDrive/deeplearning_project/arxiv_stat_pdfs"
text_dir = "/content/drive/MyDrive/deeplearning_project/arxiv_stat_text"
os.makedirs(text_dir, exist_ok=True)


for paper in os.listdir(pdf_dir):
    try:
        arxiv_id = paper
        pdf_path = os.path.join(pdf_dir, f"{arxiv_id}")
        text = extract_text_from_pdf(pdf_path)
        text_path = os.path.join(text_dir, f"{arxiv_id}.json")
        with open(text_path, "w") as f:
            json.dump({"text": text}, f)
    except Exception as e:
        print(f"Error processing {arxiv_id}: {e}")
